### Code implements newtonian optimization on a surface

In [ ]:
from sympy import symbols, Eq, sin, solve, diff, lambdify
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
import random

In [ ]:
def count_calls(func):
    def wrapper(*args, **kwargs):
        wrapper.calls += 1
        return func(*args, **kwargs)
    wrapper.calls = 0
    return wrapper

In [ ]:
# Define the symbols
x1, x2 = symbols('x1 x2')
variables = [x1, x2]

# Original equation
eq1 = Eq(sin(2*x1 - x2) - 1.2*x1, 1.52)
eq2 = Eq(0.8 * x1 + 1.5 * x2 , 1.83)


eq1_transformed = Eq(eq1.lhs - eq1.rhs, 0)
eq2_transformed = Eq(eq2.lhs - eq2.rhs, 0)

cost_function = eq1_transformed.lhs**2 + eq2_transformed.lhs**2
cost_function_lambda = count_calls(lambdify((x1, x2), cost_function, "numpy"))
cost_function

In [ ]:
def fig_config(fig):
    fig.update_layout(title='Cost Function', autosize=False,
                    width=500, height=500,
                    margin=dict(l=65, r=50, b=65, t=90),
                    scene=dict(
                            xaxis_title='x1',
                            yaxis_title='x2',
                            zaxis_title='Cost'))


In [ ]:
# Generate x1 and x2 values
x1_values = np.linspace(-15, 15, 600)
x2_values = np.linspace(-15, 15, 600)

# Generate a grid of points
x1_grid, x2_grid = np.meshgrid(x1_values, x2_values)

# Compute the cost function at each point on the grid
Z = cost_function_lambda(x1_grid, x2_grid)

In [ ]:
fig = go.Figure(data=[go.Surface(z=Z, x=x1_grid, y=x2_grid)])
fig_config(fig)
fig.show()

In [ ]:
a0 = [14, 12]
a1 = [-10, 10]

In [ ]:
fig = go.Figure(data=[go.Surface(z=Z, x=x1_grid, y=x2_grid)])
fig.add_trace(go.Scatter3d(x=[a0[0]], y=[a0[1]], z=[cost_function_lambda(a0[0], a0[1])], mode='markers', name='a0'))
fig.add_trace(go.Scatter3d(x=[a1[0]], y=[a1[1]], z=[cost_function_lambda(a1[0], a1[1])], mode='markers', name='a1'))
fig_config(fig)
fig.show()

In [ ]:
def dichotomy_optimization(f, a, b, tolerance=1e-5, max_iterations=100):
    if a >= b:
        raise ValueError("Invalid interval: 'a' must be less than 'b'.")
    if tolerance <= 0:
        raise ValueError("Tolerance must be positive.")
    
    iteration = 0
    while (b - a) / 2 > tolerance and iteration < max_iterations:
        c = (a + b) / 2

        epsilon = (b - a) / 4
        left = c - epsilon
        right = c + epsilon
        
        f_left = f(left)
        f_right = f(right)

        if f_left < f_right:
            b = c  
        else:
            a = c 
        
        iteration += 1
    
    return (a + b) / 2


In [ ]:
def golden_ratio_optimization(f, a, b, tolerance=1e-5, max_iterations=100):
    if a >= b:
        raise ValueError("Invalid interval: 'a' must be less than 'b'.")
    if tolerance <= 0:
        raise ValueError("Tolerance must be positive.")
    
    # The golden ratio
    phi = (1 + 5 ** 0.5) / 2
    
    # Set the initial points
    c = b - (b - a) / phi
    d = a + (b - a) / phi
    
    iteration = 0
    while (b - a) > tolerance and iteration < max_iterations:
        # Evaluate the function at the points c and d
        fc = f(c)
        fd = f(d)
        
        # Narrow the interval
        if fc < fd:
            b = d
            d = c
            c = b - (b - a) / phi
        else:
            a = c
            c = d
            d = a + (b - a) / phi
        
        iteration += 1
    
    # The point where f(x) is minimized is in the middle of the final interval
    return (b + a) / 2


In [ ]:
def coordinate_descent_step(current_point):
    fixed_axis_ind = random.choice([0, 1])
    working_axis_ind = (fixed_axis_ind + 1) % 2
    print(f"\tWoking axis: {variables[working_axis_ind]}")

    # making our function one dimensional
    cost_function_fixed = cost_function.subs(variables[fixed_axis_ind], current_point[fixed_axis_ind])
    # evaluating the derivative at the current point
    derivative_value = diff(cost_function_fixed, variables[working_axis_ind]).subs(
        variables[working_axis_ind], current_point[working_axis_ind]).evalf()
    
    get_new_point = lambda step_size: [float(current_point[i] - step_size * derivative_value) if i == working_axis_ind else float(current_point[i]) for i in range(len(current_point))]

    step_function_lambda = lambda l: cost_function_lambda(*get_new_point(l))
    step_size = dichotomy_optimization(step_function_lambda, -10, 10, 0.01)

    new_point = get_new_point(step_size)

    return new_point

In [ ]:
def gradient_descent_step(initial_point):
    var_value_dict = {var_name: initial_point[i] for i, var_name in enumerate(variables)}
    gradient = [diff(cost_function, var).subs(var_value_dict).evalf() for var in variables]
    # print('Gradient:', gradient)
    
    get_new_point = lambda step_size: [float(initial_point[i] - step_size * gradient[i]) for i in range(len(initial_point))]
    step_function_lambda = lambda l: cost_function_lambda(*get_new_point(l))
    
    step_size = golden_ratio_optimization(step_function_lambda, -10, 10, 0.01)
    new_point= get_new_point(step_size)
    return new_point

In [ ]:
def optimization(step_function, initial_point):
    prev_point = initial_point.copy()
    prev_point_cost = cost_function_lambda(*prev_point)
    counter = 0
    while True:
        current_point = step_function(prev_point)
        cost = cost_function_lambda(*current_point)
        
        if abs(prev_point_cost - cost) < 1e-9:
            break
        prev_point = current_point.copy()
        prev_point_cost = cost     
        fig.add_trace(go.Scatter3d(x=[current_point[0]], y=[current_point[1]], z=[cost], mode='markers', name=f'a0_{counter}'))
        print(f'Iteration {counter}:')
        print(f'\tCurrent point: {current_point}')
        print(f'\tCost: {cost}')
        counter+=1
    
    return current_point



Coordinate descent with random axis selection + Dichotomy method for selecting step size

In [ ]:
# A0
cost_function_lambda.calls = 0
fig = go.Figure(data=[go.Surface(z=Z, x=x1_grid, y=x2_grid)])
fig.add_trace(go.Scatter3d(x=[a0[0]], y=[a0[1]], z=[
              cost_function_lambda(a0[0], a0[1])], mode='markers', name='a0'))
optimization(coordinate_descent_step, a0)
fig_config(fig)
fig.show()
print(cost_function_lambda.calls)


In [ ]:
cost_function_lambda.calls = 0
fig = go.Figure(data=[go.Surface(z=Z, x=x1_grid, y=x2_grid)])
fig.add_trace(go.Scatter3d(x=[a1[0]], y=[a1[1]], z=[
              cost_function_lambda(a1[0], a1[1])], mode='markers', name='a0'))
optimization(coordinate_descent_step, a1)
fig_config(fig)
fig.show()
print(cost_function_lambda.calls)

Gradient descent + golden ratio method

In [ ]:
cost_function_lambda.calls = 0
fig = go.Figure(data=[go.Surface(z=Z, x=x1_grid, y=x2_grid)])
fig.add_trace(go.Scatter3d(x=[a0[0]], y=[a0[1]], z=[
              cost_function_lambda(a0[0], a0[1])], mode='markers', name='a0'))
optimization(gradient_descent_step, a0)
fig_config(fig)
fig.show()
print(cost_function_lambda.calls)

In [ ]:
cost_function_lambda.calls = 0
fig = go.Figure(data=[go.Surface(z=Z, x=x1_grid, y=x2_grid)])
fig.add_trace(go.Scatter3d(x=[a1[0]], y=[a1[1]], z=[
              cost_function_lambda(a1[0], a1[1])], mode='markers', name='a0'))
optimization(gradient_descent_step, a1)
fig_config(fig)
fig.show()
print(cost_function_lambda.calls)

In [ ]:
from scipy.optimize import minimize

# Define the function
def f(x):
    return (np.sin(2*x[0]-x[1])-1.2*x[0]-1.52)**2 + (0.8*x[0]+1.5*x[1]-1.83)**2

# Initial guess
x0 = np.array([-1, 1])

# Call the minimize function
res = minimize(f, x0)

print(f"The minimum of the function is at x = {res.x}")
print(f"The value of the function at this point is {res.fun}")